In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [86]:
from subgrounds.subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField
from datetime import datetime

sg = Subgrounds()

sushi = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/sushiswap/exchange')

In [92]:
liqPosSnaps = sushi.Query.liquidityPositionSnapshots(
    # first = 1000,
    orderBy = 'timestamp',
    orderDirection = 'desc',
    where = {
        'liquidityTokenBalance_gt': 0
    }
)

pairs = sushi.Query.pairs

## Synthetic field to transfer timestamp to datetime
sushi.LiquidityPositionSnapshot.datetime = SyntheticField(
    f = lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    type_ = SyntheticField.STRING,
    deps = sushi.LiquidityPositionSnapshot.timestamp
)

req = sg.mk_request([
    liqPosSnaps.datetime,
    # liqPosSnaps.timestamp,
    liqPosSnaps.user.id,
    liqPosSnaps.pair.id,
    liqPosSnaps.liquidityTokenBalance
])

print(req.graphql)

query {
  x75e638e15855ad02: liquidityPositionSnapshots(orderBy: timestamp, orderDirection: desc, where: {liquidityTokenBalance_gt: "0.0"}) {
    datetime
    user {
      id
    }
    pair {
      id
    }
    liquidityTokenBalance
  }
}


In [93]:
[pairs, lp_snaps] = sg.query_df([ 
    pairs.id,
    pairs.name,
    liqPosSnaps.datetime,
    # liqPosSnaps.timestamp,
    liqPosSnaps.user.id,
    liqPosSnaps.pair.id,
    liqPosSnaps.liquidityTokenBalance
])


In [94]:
pairs.head()

,pairs_id,pairs_name
0,0x00040a7ebfc9f6fbce4d23bd66b79a603ba1c323,TUSD-Xi
1,0x00088e1f7510370fab86a7bd10578b578c61c723,LON-SUSHI
2,0x0013df4ec514d5839ad4f535e4987498ebb58bf5,ORANGE-WETH
3,0x0018fb451a46ae397b8569936bc5bb5ff03cfd18,BEL-WETH
4,0x001b6450083e531a5a7bf310bd2c1af4247e23d4,UMA-WETH


In [95]:
lp_snaps.head()

,liquidityPositionSnapshots_datetime,liquidityPositionSnapshots_user_id,liquidityPositionSnapshots_pair_id,liquidityPositionSnapshots_liquidityTokenBalance
0,2022-04-03 18:40:10,0xe98477bdc16126bb0877c6e3882e3edd72571cc2,0x6a091a3406e0073c3cd6340122143009adac0eda,10561.220710
1,2022-04-03 18:40:10,0x5ad6211cd3fde39a9cecb5df6f380b8263d1e277,0x130f4322e5838463ee460d5854f5d472cfc8f253,0.118541
2,2022-04-03 18:38:41,0x5ad6211cd3fde39a9cecb5df6f380b8263d1e277,0x5fa4370164a2fabeef159b893299d59ff5dc1e6d,90.371186
3,2022-04-03 18:37:29,0xfa29484aa6b57b93748baf20fb9ed2bd46e11f5d,0x6a091a3406e0073c3cd6340122143009adac0eda,5.986459
4,2022-04-03 18:37:29,0x5ad6211cd3fde39a9cecb5df6f380b8263d1e277,0x6a091a3406e0073c3cd6340122143009adac0eda,0.506177
